### 错误、调试和测试

程序编写，用户输入，运行过程

### 错误处理

错误代码：-1，容易与正常结果混在一起

In [1]:
def foo():
    r = some_function()
    if r == (-1):
        return (-1)
    # do something
    return r

def bar():
    r = foo()
    if r == (-1):
        return 'Error'
    else:
        pass
# 一旦出错还要一级级上报

### try...except...finally...

In [4]:
try:
    print('try...')
    r = 10 / 0
    print('result:', r)
except ValueError as e:
    print('ValueError:', e)
except ZeroDivisionError as e:
    print 'except:', e
else:
    print 'no error!'
finally:
    print('finally...')
print('END')

try...
except: integer division or modulo by zero
finally...
END


所有的错误都是从BaseException类派生的https://docs.python.org/3/library/exceptions.html#exception-hierarchy

使用try...except捕获错误还有一个巨大的好处，就是可以跨越多层调用，比如函数main()调用foo()，foo()调用bar()，结果bar()出错了，这时，只要main()捕获到了，就可以处理：

#### 调用堆栈

如果错误没有被捕获，它就会一直往上抛，最后被Python解释器捕获，打印一个错误信息，然后程序退出。

In [6]:
'''
$ python3 err.py
Traceback (most recent call last):
  File "err.py", line 11, in <module>
    main()
  File "err.py", line 9, in main
    bar('0')
  File "err.py", line 6, in bar
    return foo(s) * 2
  File "err.py", line 3, in foo
    return 10 / int(s)
ZeroDivisionError: division by zero
'''


'\n$ python3 err.py\nTraceback (most recent call last):\n  File "err.py", line 11, in <module>\n    main()\n  File "err.py", line 9, in main\n    bar(\'0\')\n  File "err.py", line 6, in bar\n    return foo(s) * 2\n  File "err.py", line 3, in foo\n    return 10 / int(s)\nZeroDivisionError: division by zero\n'

#### 记录错误

如果不捕获错误，自然可以让Python解释器来打印出错误堆栈，但程序也被结束了。既然我们能捕获错误，就可以把错误堆栈打印出来，然后分析错误原因，同时，让程序继续执行下去。

logging

In [7]:
import logging

def foo(s):
    return 10 / int(s)

def bar(s):
    return foo(s) * 2

def main():
    try:
        bar('0')
    except Exception as e:
        logging.exception(e)
        
main()
print 'End'

ERROR:root:integer division or modulo by zero
Traceback (most recent call last):
  File "<ipython-input-7-83fed479afcd>", line 11, in main
    bar('0')
  File "<ipython-input-7-83fed479afcd>", line 7, in bar
    return foo(s) * 2
  File "<ipython-input-7-83fed479afcd>", line 4, in foo
    return 10 / int(s)
ZeroDivisionError: integer division or modulo by zero


End


通过配置，logging还可以把错误记录到日志文件里，方便事后排查。

#### 抛出错误

如果要抛出错误，首先根据需要，可以定义一个错误的class，选择好继承关系，然后，用raise语句抛出一个错误的实例：

In [9]:
# 自定义错误
class FooError(ValueError):
    pass

def foo(s):
    n = int(s)
    if n == 0:
        raise FooError('invalid value: %s' % s)
    return 10 / n

foo(0)

FooError: invalid value: 0

在bar()函数中，我们明明已经捕获了错误，但是，打印一个ValueError!后，又把错误通过raise语句抛出去了

In [12]:

def foo(s):
    n = int(s)
    if n==0:
        raise ValueError('invalid value: %s' % s)
    return 10 / n

def bar():
    try:
        foo('0')
    except ValueError as e:
        print('ValueError!')
        raise

bar()

ValueError!


ValueError: invalid value: 0

捕获错误目的只是记录一下，便于后续追踪。但是，由于当前函数不知道应该怎么处理该错误，所以，最恰当的方式是继续往上抛，让顶层调用者去处理。

raise语句如果不带参数，就会把当前错误原样抛出。此外，在except中raise一个Error，还可以把一种类型的错误转化成另一种类型：

In [11]:
try:
    10 / 0
except ZeroDivisionError:
    raise ValueError('input error!')

ValueError: input error!

只要是合理的转换逻辑就可以，但是，决不应该把一个IOError转换成毫不相干的ValueError。

#### 小结


出错时，会分析错误信息并定位错误发生的代码位置才是最关键的。

程序也可以主动抛出错误，让调用者来处理相应的错误。但是，应该在文档中写清楚可能会抛出哪些错误，以及错误产生的原因。

### 调试

1、print()，会使程序包含很多垃圾信息
2、assert n != 0, 'n is zero',缺点同上，但可在启动Python解释器时，使用-o关闭，相当于pass

#### 3、logging

In [16]:
import logging
logging.basicConfig(level=logging.INFO)
# logging.basicConfig(filename='', format='%(asctime)s %(message)s', level=logging.DEBUG)

s = '0'
n = int(s)
logging.info('n = %d' % n)
print 10 / n

ZeroDivisionError: integer division or modulo by zero

logging的好处，它允许你指定记录信息的级别，有debug，info，warning，error等几个级别，当我们指定level=INFO时，logging.debug就不起作用了。同理，指定level=WARNING后，debug和info就不起作用了。

logging的另一个好处是通过简单的配置，一条语句可以同时输出到不同的地方，比如console和文件。

#### 4、pdb

以python3 -m pdb err.py启动后，pdb定位到下一步要执行的代码-> s = '0'。输入命令l来查看代码：

输入命令n可以单步执行代码，任何时候都可以输入命令p 变量名来查看变量，输入命令q结束调试，退出程序，太麻烦。。

#### 5、pdb.set_trace()

这个方法也是用pdb，但是不需要单步执行，我们只需要import pdb，然后，在可能出错的地方放一个pdb.set_trace()，就可以设置一个断点：

In [ ]:
import pdb

s = '0'
n = int(s)
pdb.set_trace() # 运行到此自动停止
print(10 / n)

--Return--
> <ipython-input-18-44f8cf924a10>(5)<module>()->None
-> pdb.set_trace() # 运行到此自动停止


运行代码，程序会自动在pdb.set_trace()暂停并进入pdb调试环境，可以用命令p查看变量，或者用命令c继续运行：

效率低

#### 6、IDE

#### 最后你会发现logging才是最终武器

### 单元测试

“测试驱动开发”（TDD：Test-Driven Development）

In [ ]:
# 确定需求
# 编写测试代码
# 编写代码
class Dict(dict):
    
    def __init__(self, **kw):
        super().__init__(**kw)
        
    def __getattr__(self, key):
        try:
            return self[key]
        except KeyError:
            raise AttributeError(r"'Dict' objecthas no attribute '%s" % key)
            
    def __setatr__(self, key, value):
        self[key] = value
        

# 编写单元测试 unittest模块
import unittest

# from mydict import Dict

class TestDict(unittest.TestCase): # 测试类，从unittest.TestCase继承
    
    def test_init(self):
        d = Dict(a=1, b='test')
        self.assertEqual(d.a, 1)
        self.assertEqual(d.b, 'test')
        self.assertTrue(isinstance(d, dict))
        
    def test_key(self):
        d = Dict()
        d['key'] = 'value'
        aelf.assertEqual(d.key, 'value')
        
    def test_attr(self):
        d = Dict()
        with self.assertRaise(KeyError):
            value = d['empty']
            
    def test_attrerror(self):
        d = Dict()
        with self.assertRaises(AttributeError):
            value = d.empty

self.assertEqual(abs(-1), 1) # 断言函数返回的结果与1相等


In [ ]:
'''
另一种重要的断言就是期待抛出指定类型的Error，
比如通过d['empty']访问不存在的key时，断言会抛出KeyError
with self.assertRaises(KeyError):
    value = d['empty']
    
而通过d.empty访问不存在的key时，我们期待抛出AttributeError：
with self.assertRaises(AttributeError):
    value = d.empty

'''

#### 运行单元测试

In [ ]:
'''
简单的运行方式是在mydict_test.py的最后加上两行代码：
if __name__ == '__main__':
    unittest.main()
    
这样就可以把mydict_test.py当做正常的python脚本运行：
$ python3 mydict_test.py

另一种方法是在命令行通过参数-m unittest直接运行单元测试：
$ python3 -m unittest mydict_test
'''

#### setUp与tearDown

可以在单元测试中编写两个特殊的setUp()和tearDown()方法。这两个方法会分别在每调用一个测试方法的前后分别被执行。

设想你的测试需要启动一个数据库，这时，就可以在setUp()方法中连接数据库，在tearDown()方法中关闭数据库，这样，不必在每个测试方法中重复相同的代码：

In [ ]:
class TestDict(unittest.TestCase):
    
    def setUp(self):
        print 'setUp...'
        
    def tearDown(self):
        print 'tearDown...'
# 每个测试方法调用前后是否会打印出setUp...和tearDown...

单元测试的测试用例要覆盖常用的输入组合、边界条件和异常。

### 文档测试

In [ ]:
'''
当我们编写注释时，如果写上这样的注释：
def abs(n):
    三引号
    Function to get absolute value of number.

    Example:

    >>> abs(1)
    1
    >>> abs(-1)
    1
    >>> abs(0)
    0
    三引号
    return n if n >= 0 else (-n)
    
    Python内置的“文档测试”（doctest）模块可以直接提取注释中的代码并执行测试。

doctest严格按照Python交互式命令行的输入和输出来判断测试结果是否正确。只有测试异常的时候，可以用...表示中间一大段烦人的输出。
'''

In [ ]:
class Dict(dict): # ？
    '''
    Simple dict but also support access as x.y style.

    >>> d1 = Dict()
    >>> d1['x'] = 100
    >>> d1.x
    100
    >>> d1.y = 200
    >>> d1['y']
    200
    >>> d2 = Dict(a=1, b=2, c='3')
    >>> d2.c
    '3'
    >>> d2['empty']
    Traceback (most recent call last):
        ...
    KeyError: 'empty'
    >>> d2.empty
    Traceback (most recent call last):
        ...
    AttributeError: 'Dict' object has no attribute 'empty'
    '''
    def __init__(self, **kw):
        super(Dict, self).__init__(**kw)

    def __getattr__(self, key):
        try:
            return self[key]
        except KeyError:
            raise AttributeError(r"'Dict' object has no attribute '%s'" % key)

    def __setattr__(self, key, value):
        self[key] = value

if __name__=='__main__':
    import doctest
    doctest.testmod()
    
# doctest.testmod()

注意到最后3行代码。当模块正常导入时，doctest不会被执行。只有在命令行直接运行时，才执行doctest。所以，不必担心doctest会在非测试环境下执行。

doctest非常有用，不但可以用来测试，还可以直接作为示例代码。通过某些文档生成工具，就可以自动把包含doctest的注释提取出来。用户看文档的时候，同时也看到了doctest。